# Day_08: Filing and tracking issues in GitHub

Today I ran into some strange behavior when I tried to generalize my Lagrangian equations of motion. It boils down to something happening with Symbolic `Vectors`

Take for example

In [1]:
using Symbolics

@variables t x[1:2](t) # independent and dependent variables
@variables k # parameters

Dx1 = Differential(x[1])
Dx2 = Differential(x[2])

V = 1/2*k*(x[2]- x[1])^2

0.5k*((x[2](t) - x[1](t))^2)

I define a symbolic function `V` that is

$\frac{1}{2}\left(x_2 - x_1\right)^2$. 

Now, I take a derivative with respect to $x_1$

$\frac{\partial L}{\partial x_1} = -k(x_2 - x_1)$

but instead I get

In [2]:
Symbolics.expand_derivatives(Dx1(V))

0

_Weird!_

Expanding the polynomial there are 3 terms
- $1/2 kx_1^2$
- $1/2 kx_2^2$
- $-kx_1x_2$

So, I'll take derivatives of these terms and see what I get

In [3]:
Vterms = [1/2*k*x[1]^2,1/2*k*x[2]^2, -k*x[1]*x[2]]

3-element Vector{Num}:
                  0.5k*(x[1](t)^2)
                  0.5k*(x[2](t)^2)
 -k*x[1](t)*x[2](t)

In [4]:
Symbolics.expand_derivatives.(Dx1.(Vterms))

3-element Vector{Num}:
              k*x[1](t)
              k*x[2](t)
 -k*x[1](t) - k*x[2](t)

Here is the root of my problem, for some reason the partial derivative `Differential(x[1])` is treating `x[1]` and `x[2]` as the same variable. The result should have been

- $kx_1$
- $0$
- $-kx_2$


## Unindexed variables work
Now, I can compare this to an unindexed set of variables

In [5]:
@variables t y1(t) y2(t) # independent and dependent variables
@variables k # parameters

Dy1 = Differential(y1)
Dy2 = Differential(y2)

V = 1/2*k*(y2- y1)^2

0.5k*((y2(t) - y1(t))^2)

In [6]:
Symbolics.expand_derivatives(Dy1(V))

0.5k*(2y1(t) - 2y2(t))

This returns the correct derivative, 

$\frac{\partial V}{\partial y_1} = -k(y_2 - y_1)$

In [7]:
@variables z_1(t)

1-element Vector{Num}:
 z_1(t)

In [8]:
D = Differential(t)

(::Differential) (generic function with 2 methods)

In [9]:
D.(x)

(broadcast(Differential(t), map(Symbolics.CallWith((t,)), x)))[1:2]

In [10]:
D.([y1,y2])

2-element Vector{Num}:
 Differential(t)(y1(t))
 Differential(t)(y2(t))

## Symbolics still adding support for arrays

The Symbolics.jl package is incredible. Its still working on supporting differentiation with arrays. I filed this behavior on [github.com/Symbolics.jl issue #571](https://github.com/JuliaSymbolics/Symbolics.jl/issues/571). From what I can see, the main driver behind symbolic differentiation is in the [`diff.jl`](https://github.com/JuliaSymbolics/Symbolics.jl/blob/master/src/diff.jl) file. 

I was getting the error from line 58

```julia
    if symtype(expr) <: AbstractArray
        error("Differentiation of expressions involving arrays and array variables is not yet supported.")
    end
```

but [@shashi](https://github.com/shashi) added some great exceptions that remove this error in [#570](https://github.com/JuliaSymbolics/Symbolics.jl/pull/570) and [#568](https://github.com/JuliaSymbolics/Symbolics.jl/pull/570). There is always more work to be done on projects like this and because Julia is open source, I can dig into the code, offer help and try to add my experiences to the solution.

## Wrapping up

Today, I found some strange behavior in the Symbolics.jl package. I'll continue to update/add to the Julia projects as issues or hopefully merge some improvements. It would be great to document the process of a PR merge from start-to-finish. 

For now, the work-around appears to be that `jacobian` and `Differential` only support scalar values. So I can create a `Vector` of individual values as such

```julia
@variables x1(t) x2(t)
x = [x1, x2]
```

The development behind Julia and Symbolics.jl is active and inspiring, so I'm sure this work-around won't need to exist much longer.